In [1]:
import torch
import torch.nn as nn
from model import LocalPredictor

In [2]:
import os

In [3]:
import pickle as pk

In [4]:
import numpy as np

In [5]:
local_predictor = torch.load('./results_tokyo/local_predictor.pytorch').cuda(1)

In [6]:
date_list = [(2012, m, d) for m in range(12, 13) for d in range(1, 32)] + [(2013, m, d) for m in range(1, 3) for d in range(1, 32)]

In [7]:
T = 96
dT = 4
batch_size = 256

In [8]:
data = dict({})
for m in range(11, 12):
    for d in range(1, 29):
        filename = '/data/fan/UsersInTokyoProcessed/2012{:02d}{:02d}_interp.pk'.format(m, d)
        print(filename)
        with open(filename, 'rb') as f:
            data[(m, d)] = pk.load(f)

/data/fan/UsersInTokyoProcessed/20121101_interp.pk
/data/fan/UsersInTokyoProcessed/20121102_interp.pk
/data/fan/UsersInTokyoProcessed/20121103_interp.pk
/data/fan/UsersInTokyoProcessed/20121104_interp.pk
/data/fan/UsersInTokyoProcessed/20121105_interp.pk
/data/fan/UsersInTokyoProcessed/20121106_interp.pk
/data/fan/UsersInTokyoProcessed/20121107_interp.pk
/data/fan/UsersInTokyoProcessed/20121108_interp.pk
/data/fan/UsersInTokyoProcessed/20121109_interp.pk
/data/fan/UsersInTokyoProcessed/20121110_interp.pk
/data/fan/UsersInTokyoProcessed/20121111_interp.pk
/data/fan/UsersInTokyoProcessed/20121112_interp.pk
/data/fan/UsersInTokyoProcessed/20121113_interp.pk
/data/fan/UsersInTokyoProcessed/20121114_interp.pk
/data/fan/UsersInTokyoProcessed/20121115_interp.pk
/data/fan/UsersInTokyoProcessed/20121116_interp.pk
/data/fan/UsersInTokyoProcessed/20121117_interp.pk
/data/fan/UsersInTokyoProcessed/20121118_interp.pk
/data/fan/UsersInTokyoProcessed/20121119_interp.pk
/data/fan/UsersInTokyoProcessed

In [9]:
data_doc = dict({})

for date in data:
    for uid in data[date]:
        if uid not in data_doc:
            data_doc[uid] = [data[date][uid]]
        else:
            data_doc[uid].append(data[date][uid])

In [10]:
for uid in data_doc:
    data_doc[uid] = torch.LongTensor(data_doc[uid]).cuda(1)

In [12]:
mrr_avg_rank_dict = dict({})

for y, m, d in date_list:
    filename = '/data/fan/UsersInTokyoProcessed/{:04d}{:02d}{:02d}_interp.pk'.format(y, m, d)
    if not os.path.isfile(filename):
        continue
    print(filename)
    
    with open(filename, 'rb') as f:
        cnt_day = 0
        data = pk.load(f)
        for uid in data:
            data[uid] = torch.LongTensor([data[uid]]).cuda(1)
            
        mrr = 0.0
        avg_rank = 0.0
        acc5 = 0.0
        
        with torch.no_grad():
            for t in range(T - 2 * dT):
                
                for uid in data:
                    if np.random.ranf() < 0.05:
                        cnt_day += 1
                        data_x = data[uid][:, t: t + dT]
                        data_t = torch.zeros_like(data_x) + t
                        data_y = data[uid][:, t + 2 * dT - 1].cpu().numpy()
                        if uid in data_doc:
                            data_doc_x = data_doc[uid][:, t: t + 2 * dT]
                            data_doc_t = torch.zeros_like(data_doc_x) + t
                            pred = local_predictor.predict(data_x, data_t, data_doc_x, data_doc_t).cpu().numpy()
                        else:
                            pred = local_predictor.predict(data_x, data_t, None, None).cpu().numpy()
                            
                        rank = np.where((np.argsort(-pred, axis=1, ).T - data_y).T == 0)[1] + 1
                        mrr += np.sum(1 / rank)
                        avg_rank += np.sum(rank)
                        acc5 += np.sum(rank <= 5)
                        
            print('Evaluation: mrr = {:.4f}, avg_rank = {:.4f}, acc@5 = {:.4f}'.format(mrr / cnt_day, avg_rank / cnt_day, acc5 / cnt_day))
            mrr_avg_rank_dict[(y, m, d)] = (mrr / cnt_day, avg_rank / cnt_day, acc5 / cnt_day)

/data/fan/UsersInTokyoProcessed/20121201_interp.pk
Evaluation: mrr = 0.8127, avg_rank = 18.3622, acc@5 = 0.8702
/data/fan/UsersInTokyoProcessed/20121202_interp.pk
Evaluation: mrr = 0.8241, avg_rank = 17.5221, acc@5 = 0.8788
/data/fan/UsersInTokyoProcessed/20121203_interp.pk
Evaluation: mrr = 0.8357, avg_rank = 14.0560, acc@5 = 0.8929
/data/fan/UsersInTokyoProcessed/20121204_interp.pk
Evaluation: mrr = 0.8319, avg_rank = 14.4819, acc@5 = 0.8897
/data/fan/UsersInTokyoProcessed/20121205_interp.pk
Evaluation: mrr = 0.8289, avg_rank = 15.1371, acc@5 = 0.8875
/data/fan/UsersInTokyoProcessed/20121206_interp.pk
Evaluation: mrr = 0.8275, avg_rank = 15.4379, acc@5 = 0.8854
/data/fan/UsersInTokyoProcessed/20121207_interp.pk
Evaluation: mrr = 0.8217, avg_rank = 16.1842, acc@5 = 0.8803
/data/fan/UsersInTokyoProcessed/20121208_interp.pk
Evaluation: mrr = 0.8105, avg_rank = 19.2070, acc@5 = 0.8677
/data/fan/UsersInTokyoProcessed/20121209_interp.pk
Evaluation: mrr = 0.8238, avg_rank = 17.6665, acc@5 =

Evaluation: mrr = 0.8281, avg_rank = 19.3601, acc@5 = 0.8801
/data/fan/UsersInTokyoProcessed/20130213_interp.pk
Evaluation: mrr = 0.8260, avg_rank = 19.3076, acc@5 = 0.8791
/data/fan/UsersInTokyoProcessed/20130214_interp.pk
Evaluation: mrr = 0.8250, avg_rank = 19.5975, acc@5 = 0.8777
/data/fan/UsersInTokyoProcessed/20130215_interp.pk
Evaluation: mrr = 0.8219, avg_rank = 20.0110, acc@5 = 0.8753
/data/fan/UsersInTokyoProcessed/20130216_interp.pk
Evaluation: mrr = 0.8188, avg_rank = 22.2562, acc@5 = 0.8702
/data/fan/UsersInTokyoProcessed/20130217_interp.pk
Evaluation: mrr = 0.8312, avg_rank = 20.2217, acc@5 = 0.8814
/data/fan/UsersInTokyoProcessed/20130218_interp.pk
Evaluation: mrr = 0.8308, avg_rank = 19.3217, acc@5 = 0.8812
/data/fan/UsersInTokyoProcessed/20130219_interp.pk
Evaluation: mrr = 0.8272, avg_rank = 19.7046, acc@5 = 0.8792
/data/fan/UsersInTokyoProcessed/20130220_interp.pk
Evaluation: mrr = 0.8236, avg_rank = 19.9766, acc@5 = 0.8767
/data/fan/UsersInTokyoProcessed/20130221_in

In [13]:
with open('./results_tokyo/evaluation_local_rank.pk', 'wb') as f:
    pk.dump(mrr_avg_rank_dict, f)